# Generate temperature figures
In this notebook we
- Produce the Black summer MSE/TWMSE plot
- Calculate confidence intervals
- Evaluate the NT temperature forecasts. We don't show this figure in the paper, but we discuss the results.

In [107]:
import numpy as np
import pandas as pd
import xarray as xr

from scores.continuous import mse
from scores.processing import broadcast_and_match_nan
from scores.stats.statistical_tests import diebold_mariano

from plotly.subplots import make_subplots
import plotly.graph_objects as go

from threshold_weighed_score import threshold_weighted_squared_error

## First - analyse performance for Black Summer

### Get data for Black summer

In [108]:
# Black summer data

official_bs = xr.open_dataarray("../data/temperature/Official_MaxT_00_20191201-20200229.nc")
# Note that the 18Z PtOCF was the automated guidance available to meteorologists for the
# afternoon (00Z) official forecast issue during the 2019-2020 summer
autofcst_bs = xr.open_dataarray("../data/temperature/PtOCF_MaxT_12_20191201-20200229.nc")
obs_bs = xr.open_dataarray("../data/temperature/obs_MaxT_20191201-20200229.nc")

# Match missing data between datasets
official_bs, autofcst_bs, obs_bs = broadcast_and_match_nan(official_bs, autofcst_bs, obs_bs)

### Calculate scores

In [109]:
# threshold weighted scores for Black Summer
# load thresholds
thresholds = pd.read_json("../data/climate/max_t_0.97.json", typ="series").to_xarray()
thresholds = thresholds.rename({"index": "station_number"})

# get common stations between thresholds and the forecasts and obs
common_stations = set(thresholds.station_number.values).intersection(obs_bs.station_number.values)
obs_bs = obs_bs.sel(station_number=list(common_stations))
official_bs = official_bs.sel(station_number=list(common_stations))
autofcst_bs = autofcst_bs.sel(station_number=list(common_stations))
thresholds = thresholds.sel(station_number=list(common_stations))
inf_threshold = thresholds * np.inf

# Calculate threshold weighted MSE
official_bs_tw = threshold_weighted_squared_error(official_bs, obs_bs, interval_where_one = (thresholds, inf_threshold), dims = ["lead_day"])
autofcst_bs_tw = threshold_weighted_squared_error(autofcst_bs, obs_bs, interval_where_one = (thresholds, inf_threshold), dims = ["lead_day"])

# Calculate normal MSE
autofcst_bs_mse = mse(autofcst_bs, obs_bs, preserve_dims="lead_day")
official_bs_mse = mse(official_bs, obs_bs, preserve_dims="lead_day")

### Produce plot

In [110]:
official_line_colour = 'rgba(230,159,0,1)'
autofcst_line_colour = 'rgba(86,180,233,1)'
figure = make_subplots(rows=1, cols=2, subplot_titles=("<b>(a)</b>", "<b>(b)</b>",))
figure.update_annotations(font_size=12, xshift=-160, xanchor="left")

# Left - black summer MSE
figure.add_trace(
    go.Scatter(
        x=official_bs_mse.lead_day,
        y=official_bs_mse.values,
        line=dict(color=official_line_colour),
        name="Official",
        showlegend=False
    ), row=1, col=1
)
figure.add_trace(
    go.Scatter(
        x=autofcst_bs_mse.lead_day,
        y=autofcst_bs_mse.values,
        line=dict(color=autofcst_line_colour),
        name="Automated",
        showlegend=False
    ), row=1, col=1
)

# right - black summer TW MSE
figure.add_trace(
    go.Scatter(
        x=official_bs_tw.lead_day,
        y=official_bs_tw.values,
        line=dict(color=official_line_colour),
        name="Official",
        showlegend=False
    ), row=1, col=2
)
figure.add_trace(
    go.Scatter(
        x=autofcst_bs_tw.lead_day,
        y=autofcst_bs_tw.values,
        line=dict(color=autofcst_line_colour),
        name="Automated",
        showlegend=False
    ), row=1, col=2
)

figure.update_layout(
    legend=dict(x=0.01,y=0.99),
    height=300,
    width=800,
    margin=go.layout.Margin(
        l=20, #left margin
        r=20, #right margin
        b=20, #bottom margin
        t=20, #top margin
    ),
)
figure.update_xaxes(title_text="Lead day", row=1, col=1, tickmode="linear", tick0=0, dtick=1)
figure.update_xaxes(title_text="Lead day", row=1, col=2, tickmode="linear", tick0=0, dtick=1)
figure.update_yaxes(title_text="MSE (°C<sup>2</sup>)", row=1, col=1)
figure.update_yaxes(title_text="Threshold weighted MSE (°C<sup>2</sup>)", row=1, col=2)

figure.add_annotation(x=4, y=2.5, text="Lower scores better", showarrow=False, row=1, col=1)

figure.add_annotation(x=4, y=0.4, text="Lower scores better", showarrow=False, row=1, col=2)


figure.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="right",
    x=0.99
))

In [111]:
figure.write_image("../figures/temperature.svg")

### Confidence intervals

In [112]:
# Black Summer
autofcst_bs_mse = mse(autofcst_bs, obs_bs, preserve_dims=["lead_day", "valid_15z_date"])
official_bs_mse = mse(official_bs, obs_bs, preserve_dims=["lead_day", "valid_15z_date"])
# Difference between Official and the hindcast
diff= autofcst_bs_mse - official_bs_mse
diff = diff.assign_coords(h=("lead_day", [2, 3, 4, 5, 6, 7, 8]))
dm_result = diebold_mariano(diff, "lead_day", "h")
dm_result

<xarray.Dataset>
Dimensions:          (lead_day: 7)
Coordinates:
  * lead_day         (lead_day) int64 1 2 3 4 5 6 7
Data variables:
    mean             (lead_day) float64 0.07002 0.06043 ... 0.09111 -0.02496
    dm_test_stat     (lead_day) float64 1.762 1.423 1.383 ... 0.5641 -0.712
    timeseries_len   (lead_day) int64 91 91 91 91 91 91 91
    confidence_gt_0  (lead_day) float64 0.961 0.9226 0.9167 ... 0.7137 0.2382
    ci_upper         (lead_day) float64 0.1479 0.1437 0.1684 ... 0.4076 0.04375
    ci_lower         (lead_day) float64 -0.007872 -0.02283 ... -0.2254 -0.09368

In [113]:
# Black Summer TW
official_bs_tw = threshold_weighted_squared_error(official_bs, obs_bs, interval_where_one = (thresholds, inf_threshold), dims = ["lead_day", "valid_15z_date"])
autofcst_bs_tw = threshold_weighted_squared_error(autofcst_bs, obs_bs, interval_where_one = (thresholds, inf_threshold), dims = ["lead_day", "valid_15z_date"])
# Difference between Official and the hindcast
diff= autofcst_bs_tw - official_bs_tw
diff = diff.assign_coords(h=("lead_day", [2, 3, 4, 5, 6, 7, 8]))
dm_result = diebold_mariano(diff, "lead_day", "h")
dm_result

<xarray.Dataset>
Dimensions:          (lead_day: 7)
Coordinates:
  * lead_day         (lead_day) int64 1 2 3 4 5 6 7
Data variables:
    mean             (lead_day) float64 0.06137 0.05551 ... 0.06739 -0.004441
    dm_test_stat     (lead_day) float64 1.991 2.04 2.236 ... 1.084 -0.3778
    timeseries_len   (lead_day) int64 91 91 91 91 91 91 91
    confidence_gt_0  (lead_day) float64 0.9767 0.9793 0.9873 ... 0.8608 0.3528
    ci_upper         (lead_day) float64 0.1218 0.1088 0.1533 ... 0.1893 0.0186
    ci_lower         (lead_day) float64 0.0009431 0.002187 ... -0.05448 -0.02748

# NT example

In [114]:
# NT 2016 data

official_nt_2016 = xr.open_dataarray("../data/temperature/Official_MaxT_00_20160501-20260930_NT.nc")
# Note that the 18Z PtOCF was the automated guidance available to meteorologists for the
# afternoon (00Z) official forecast issue during 2016
autofcst_nt_2016 = xr.open_dataarray("../data/temperature/PtOCF_MaxT_12_20160501-20260930_NT.nc")
obs_nt_2016 = xr.open_dataarray("../data/temperature/obs_MaxT_20160501-20160930_NT.nc")

# Match missing data between datasets
official_nt_2016, autofcst_nt_2016, obs_nt_2016 = broadcast_and_match_nan(official_nt_2016, autofcst_nt_2016, obs_nt_2016)

# Produce sample climatological forecast
nt_2016_clim_fcst = obs_nt_2016.mean("valid_15z_date") * (obs_nt_2016 * 0 + 1)

# NT 2020 data

official_nt_2020 = xr.open_dataarray("../data/temperature/Official_MaxT_00_20200501-20200930_NT.nc")
# Note that the 12Z AutoFcst was the automated guidance available to meteorologists for the
# afternoon (00Z) official forecast issue during the 2020 dry season
autofcst_nt_2020 = xr.open_dataarray("../data/temperature/AutoFcst_MaxT_12_20200501-20200930_NT.nc")
obs_nt_2020 = xr.open_dataarray("../data/temperature/obs_MaxT_20200501-20200930_NT.nc")

# Match missing data between datasets
official_nt_2020, autofcst_nt_2020, obs_nt_2020 = broadcast_and_match_nan(official_nt_2020, autofcst_nt_2020, obs_nt_2020)

# Produce sample climatological forecast
nt_2020_clim_fcst = obs_nt_2020.mean("valid_15z_date") * (obs_nt_2020 * 0 + 1)

In [115]:
# 2016 NT MSE
autofcst_2016_mse = mse(autofcst_nt_2016, obs_nt_2016, preserve_dims="lead_day")
official_2016_mse = mse(official_nt_2016, obs_nt_2016, preserve_dims="lead_day")
clim_2016_mse = mse(nt_2016_clim_fcst, obs_nt_2016, preserve_dims="lead_day")
autofcst_2016_ss = 1 - (autofcst_2016_mse /clim_2016_mse)
official_2016_ss = 1 - (official_2016_mse /clim_2016_mse)

# 2020 NT MSE
autofcst_2020_mse = mse(autofcst_nt_2020, obs_nt_2020, preserve_dims="lead_day")
official_2020_mse = mse(official_nt_2020, obs_nt_2020, preserve_dims="lead_day")
clim_2020_mse = mse(nt_2016_clim_fcst, obs_nt_2016, preserve_dims="lead_day")
autofcst_2020_ss = 1 - (autofcst_2020_mse /clim_2020_mse)
official_2020_ss = 1 - (official_2020_mse /clim_2020_mse)

In [117]:
official_line_colour = 'rgba(230,159,0,1)'
autofcst_line_colour = 'rgba(86,180,233,1)'
figure = make_subplots(rows=1, cols=2, subplot_titles=("<b>(a)</b>", "<b>(b)</b>"))
figure.update_annotations(font_size=12, xshift=-160, xanchor="left")

# Upper left - NT MSE 2016
figure.add_trace(
    go.Scatter(
        x=official_2016_ss.lead_day,
        y=official_2016_ss.values,
        line=dict(color=official_line_colour),
        name="Official"
    ), row=1, col=1
)
figure.add_trace(
    go.Scatter(
        x=autofcst_2016_ss.lead_day,
        y=autofcst_2016_ss.values,
        line=dict(color=autofcst_line_colour),
        name="Automated"
    ), row=1, col=1
)

# Upper right - NT MSE 2020
figure.add_trace(
    go.Scatter(
        x=official_2020_ss.lead_day,
        y=official_2020_ss.values,
        line=dict(color=official_line_colour),
        name="Official",
        showlegend=False
    ), row=1, col=2
)
figure.add_trace(
    go.Scatter(
        x=autofcst_2020_ss.lead_day,
        y=autofcst_2020_ss.values,
        line=dict(color=autofcst_line_colour),
        name="Automated",
        showlegend=False
    ), row=1, col=2
)


figure.update_layout(
    legend=dict(x=0.01,y=0.99),
    height=300,
    width=800,
    margin=go.layout.Margin(
        l=20, #left margin
        r=20, #right margin
        b=20, #bottom margin
        t=20, #top margin
    ),
)
figure.update_xaxes(title_text="Lead day", row=1, col=1, tickmode="linear", tick0=0, dtick=1)
figure.update_xaxes(title_text="Lead day", row=1, col=2, tickmode="linear", tick0=0, dtick=1)
figure.update_yaxes(title_text="MSE skill score", row=1, col=1)
figure.update_yaxes(title_text="MSE skill score", row=1, col=2)

figure.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99
))